# Data Cleaning

## Relevante pakker

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
from pathlib import Path # To make relative paths
pd.options.display.max_columns = None
# pd.set_option('display.max_rows', None)

### Define relevant paths
For the following code to rund you should have a project folder with the two subfolders "Code" and "Data"

In [2]:
my_path = Path.cwd() # Path to current directory
project_path = my_path.parents[0] # Path to project directory
data_path = project_path / 'Data' # Path to data folder
data_path

WindowsPath('C:/Users/nld/Introduction to Data Science/Exam Project/Data')

## Indlæser data

In [3]:
bolig_df = pd.read_csv(data_path/'boliger_salg.csv', sep = ';', encoding = 'utf-8')
print(len(bolig_df))
bolig_df

55265


C:\app\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,latitude,longitude,propertyType,priceChangePercentTotal,energyClass,openHouse,price,selfsale,rooms,size,lotSize,floor,buildYear,city,isForeclosure,isActive,municipality,zipCode,street,squaremeterPrice,area,daysForSale,createdDate,isPremiumAgent,images,net,exp,basementSize,inWatchlist,views,agentRegId,domainId,guid,agentDisplayName,groupKey,downPayment,itemType,dawaId,projectSaleUrl,additionalBuildings,lastSeen,businessArea,nonPremiumDiscrete,bfeNr,ouId,ouAddress,onTheWay,cleanStreet,otwAddress,dsAddress,boligaPlus,showLogo,randomTypeHuse
0,1844515,55.67386,12.57692,1,0,d,NaN,100000000,False,12.0,383,213,NaN,1757,København K,False,True,101,1472,Ny Kongensgade 3,261096.0,1,1008,2021-11-11T23:02:32.000Z,False,"[{'id': 1844515, 'date': '2024-08-16T15:37:06....",0,13972,105,False,94158,26472,171,628079F8-F05F-4D4D-A5B4-1AF913E20B7C,NaN,NaN,5000000,0,NaN,NaN,NaN,2024-08-15T22:39:18.433Z,80.0,False,6027332.0,0,ny-kongensgade-3-1472-koebenhavn-k,False,Ny Kongensgade,NaN,NaN,False,False,"{'leisureHouses': [], 'houses': []}"
1,2124268,55.86242,12.48589,1,0,D,NaN,33000000,False,9.0,274,22071,NaN,1934,Hørsholm,False,True,230,2970,Kirsebærvej 8,120437.0,3,38,2024-07-09T00:07:55.980Z,False,"[{'id': 2124268, 'date': '2024-08-16T15:37:06....",152039,12294,52,False,27571,17998,609,F1B6C7CC-AA4F-43AD-B744-BCEA80F993E4,NaN,NaN,1650000,0,NaN,NaN,NaN,2024-08-15T23:34:47.303Z,153.0,False,2361211.0,1399883946,kirsebaervej-8-2970-hoersholm,False,Kirsebærvej,NaN,NaN,False,False,"{'leisureHouses': [], 'houses': [[{'estateUrl'..."
2,2082241,55.76980,12.57548,1,0,C,NaN,78000000,False,12.0,456,5252,NaN,1926,Klampenborg,False,True,157,2930,Klampenborgvej 35C,171052.0,2,154,2024-03-14T23:03:35.433Z,False,"[{'id': 2082241, 'date': '2024-08-16T15:37:06....",0,37909,195,False,25796,25254,419,7A2467F3-FCA9-4D3E-98FF-2AAC6AB0D587,NaN,NaN,3900000,0,NaN,NaN,NaN,2024-08-15T22:31:01.183Z,0.0,False,2038042.0,588843670,klampenborgvej-35c-2930-klampenborg,False,Klampenborgvej,NaN,NaN,False,False,"{'leisureHouses': [], 'houses': [[{'estateUrl'..."
3,2088714,55.92461,12.44494,6,-13,J,NaN,42000000,False,9.0,375,40946,NaN,2019,Kokkedal,False,True,210,2980,Fredensborg Kongevej 42,112000.0,3,140,2024-03-28T23:02:51.050Z,False,"[{'id': 2088714, 'date': '2024-08-16T15:37:06....",0,11780,241,False,24401,26800,419,54DCADE6-8FA5-4652-B64F-65A182758CDD,NaN,NaN,2100000,0,NaN,NaN,NaN,2024-08-15T22:31:01.183Z,0.0,False,7662613.0,0,fredensborg-kongevej-42-2980-kokkedal,False,Fredensborg Kongevej,NaN,NaN,False,False,"{'leisureHouses': [], 'houses': [[{'estateUrl'..."
4,1990622,55.77462,12.46564,1,-19,E,NaN,58000000,False,14.0,704,3423,NaN,1882,Kongens Lyngby,False,True,173,2800,"Nybrovej 375, Tusculum",82386.0,2,471,2023-05-02T22:05:41.047Z,False,"[{'id': 1990622, 'date': '2024-08-16T15:37:06....",262158,54937,196,False,23296,26763,419,E3313E6F-F2AD-4C6C-834E-94BABF4ACF65,NaN,NaN,2900000,0,NaN,NaN,NaN,2024-08-15T22:31:01.183Z,0.0,False,8864741.0,2120502893,nybrovej-375-2800-kongens-lyngby,False,Nybrovej,NaN,NaN,False,False,"{'leisureHouses': [], 'houses': [[{'estateUrl'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55260,2134544,55.68054,12.58919,3,0,-,NaN,18000000,False,5.0,111,0,3.0,1670,København K,False,True,101,1254,"Lille Strandstræde 1, 3.",162162.0,1,0,2024-08-15T22:04:19.387Z,False,"[{'id': 2134544, 'date': '2024-08-16T16:28:46....",81968,3831,0,False,0,29170,1227,AA22AC07-0B93-4684-BD7C-50BE5995CB1D,NaN,NaN,900000,0,NaN,NaN,NaN,2024-08-15T22:31:01.183Z,NaN,False,126702.0,0,lille-strandstraede-1-3-1254-koebenhavn-k,False,Lille Strandstræde,NaN,NaN,False,False,NaN
55261,2134741,55.75163,12.56329,1,0,D,NaN,23995000,False,7.0,204,795,NaN,1927,Charlottenlund,False,True,157,2920,Christiansvej 38,117622.0,2,0,2024-08-15T23:30:06.690Z,False,"[{'id': 2134741, 'date': '2024-08-16T16:28:46....",0,12769,114,F

## Udvælger relevante variable

In [4]:
bolig_df.columns
bolig_df_selected = bolig_df[['id', 'latitude', 'longitude', 'propertyType', 'energyClass', 'price',
                             'rooms', 'size', 'lotSize', 'floor', 'buildYear', 'city', 'isForeclosure',
                             'municipality', 'zipCode', 'street', 'squaremeterPrice', 'area', 
                             'exp', 'basementSize', 'businessArea', 'cleanStreet', 'createdDate']]
print(len(bolig_df))

55265


## Udvælger observationer
1. Fjerner andelsboliger, fritidsboliger og fritidsgrunde
2. Vi har identificeret nogle observationer med meget lav pris, som viser sig at være timeshare, grunde i offentligt udbud mm. Vi fjerner derfor alle observationer med en pris på under 29.500
3. Fjerner dubletter


In [5]:
# 1. Fjerner andelsboliger, fritidshuse og fritidsgrunde
bolig_df_selected = bolig_df_selected[~bolig_df_selected['propertyType'].isin([4,5,8])].reset_index(drop = True)
bolig_df_selected
print(len(bolig_df[bolig_df['propertyType']==5]))
print(len(bolig_df[bolig_df['propertyType']==4]))
print(len(bolig_df[bolig_df['propertyType']==8]))
print(len(bolig_df_selected))
bolig_df_selected[bolig_df_selected['price']==10000]

723
6526
727
47289


,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate
13118,2065524,56.89526,9.22525,7,-,10000,0.0,0,703,NaN,0,Ranum,False,820,9681,Nygade 10,0.0,11,0,0,NaN,Nygade,2024-01-13T00:57:22.817Z
19117,2070806,56.85630,9.26590,7,-,10000,0.0,0,2667,NaN,0,Løgstør,False,820,9670,"Bjørnsholmvej 177, Overlade",0.0,11,0,0,NaN,Bjørnsholmvej,2024-02-06T00:57:01.653Z
20421,2065559,56.87703,9.28359,7,-,10000,0.0,0,756,NaN,0,Løgstør,False,820,9670,"Munksjørupvej 64, Overlade",0.0,11,0,0,NaN,Munksjørupvej,2024-01-13T00:57:27.373Z
22166,2065570,56.69040,9.33111,7,-,10000,0.0,0,398,NaN,0,Gedsted,False,820,9631,Vesterbro 9,0.0,11,0,0,NaN,Vesterbro,2024-01-13T00:57:28.497Z
23656,2065563,56.85478,9.26264,7,-,10000,0.0,0,1961,NaN,0,Løgstør,False,820,9670,"Mosevej 30, Overlade",0.0,11,0,0,NaN,Mosevej,2024-01-13T00:57:27.853Z
25787,2065561,56.89874,9.21992,7,-,10000,0.0,0,2106,NaN,0,Ranum,False,820,9681,Stadionvej 4,0.0,11,0,0,NaN,Stadionvej,2024-01-13T00:57:27.793Z
25788,2065571,56.67375,9.39907,7,-,10000,0.0,0,1266,NaN,0,Aalestrup,False,820,9620,"Korsvejen 77, Fjelsø",0.0,11,0,0,NaN,Korsvejen,2024-01-13T00:57:29.343Z
26697,2065569,56.71373,9.35613,7,-,10000,0.0,0,1143,NaN,0,Gedsted,False,820,9631,"Tolshøj 20, Vesterbølle",0.0,11,0,0,NaN,Tolshøj,2024-01-13T00:57:28.437Z
27338,2065560,56.90383,9.28472,7,-,10000,0.0,0,425,NaN,0,Løgstør,False,820,9670,"Kirkebakken 7, Vilsted",0.0,11,0,0,NaN,Kirkebakken,2024-01-13T00:57:27.753Z
31024,2065562,56.89670,9.22516,7,-,10000,0.0,0,614,NaN,0,Ranum,False,820,9681,Vestergade 48,0.0,11,0,0,NaN,Vestergade,2024-01-13T00:57:27.823Z


In [6]:
# 2. Fjerner boliger med pris under 29500
print(len(bolig_df_selected[bolig_df_selected['price']<=29500]))
bolig_df_selected = bolig_df_selected[bolig_df_selected['price']>29500]
print(len(bolig_df_selected))

19
47270


In [7]:
# Fjerner ejendomme med en en størrelse under 10 kvm som ikke er grunde, da det ligner erhvervsejendomme
# Filter the dataframe to remove rows where size is less than 10 and propertyType is not 7 or 8
print(len(bolig_df_selected[(bolig_df_selected['size']<10) & (~bolig_df_selected['propertyType'].isin([7]))]))
bolig_df_selected = bolig_df_selected.loc[
    (bolig_df_selected['size'] >= 10) | 
    (bolig_df_selected['propertyType'].isin([7, 8]))
]
print(len(bolig_df_selected))


40
47230


In [8]:
# Fjerner de to garager jeg har identificeret
# Løvholmen 14, st.. 10.
# Store Kongensgade 90, st.. 4.
bolig_df_selected = bolig_df_selected.loc[
    (~bolig_df_selected['street'].isin(["Løvholmen 14, st.. 10.", "Store Kongensgade 90, st.. 4."]))
]
print(len(bolig_df_selected))



47228


## Data Cleaning
Data cleaning refers to the process of identifying and correcting errors, inconsistencies, and missing values in a dataset. It is the process of making sure that the data is accurate and consistent before it is used for analysis. This includes tasks such as dealing with missing values, removing duplicate data, and correcting errors in the data. Data cleaning is an essential step in the process of working with data because it ensures that the data is of high quality and can be used to make accurate and reliable conclusions.

Nedenfor cleaner vi data ved at fjerne eventuelle dubletter og korrigere fejlværdier i variablene

## Dubletter 

### Tjekker for dubletter

In [13]:
# Tjekker for dubletter i alle kolonner
print("Dubletter:" + str(len(bolig_df[bolig_df.duplicated()])))

# Step 1: Group by 'street' and 'zipCode' and count occurrences
grouped_df = bolig_df_selected.groupby(['street', 'zipCode', 'propertyType']).size().reset_index(name='count')

# Step 2: Filter for groups where there is more than one occurrence
duplicates_df = grouped_df[grouped_df['count'] > 1]

# Step 3: Merge with original DataFrame to get all the original columns for the filtered rows
result_df = bolig_df_selected.merge(duplicates_df[['street', 'zipCode', 'propertyType']], 
                           on=['street', 'zipCode', 'propertyType'], how='inner')

pd.set_option('display.max_rows', 20)
result_df
result_df.to_csv('dubletter.csv', sep=';', index=False, encoding='utf-8')

# Der er nogle dubletter:
# 1. Villaer: Ligner fejl, nogle boliger ligger hos to mæglere. Hvis jeg kun kan finde den et sted 
# Er det den ældste som passer på det der ligger hos boliga, der er også nogle tilfælde, 
# Hvor nogle variable mangler på det nyeste men ikke den ældste række for ejendomme

# 2. Rækkehuse: Ligner også fejl. Nogle boliger ligger hos to mæglere. 
# Der er dog også et tilfælde af projektbolig, som ligger med samme adresse men to forskellige boliger
# i to forskellige størrelser. (kan evt. håndteres ved også at betinge på size, når dubletter fjernes)

# 3. Lejligheder: ligner fejl, der er nogle som ligger hos to mæglere. Igen er der en ejendom som ligger der
# 3 gange med samme adresse, men det er rækkehus eller noget og der er tre forskellige størrelser
# Kan håndteres ved at betinge på size, når dubletter fjernes

# 4. Fritidsejendom: Ligner også fejl. Igen nogle ejendomme på samme adresse med forskellig areal. 
# Kan håndteres ved at betinge på size når der fjernes dubletter
# I de tilfælde hvor jeg kun kan finde bolig hos en mægler ligner det at den ældste række passer.

# 6. Landejendom: Jeg tror mange af dem er reelle. Men det er svært. Det ligner at det er sælges en 
# ejendom og jord (dvs. flere matrikler). Jeg tror altid at de vil blive solgt sammen. 
# Jeg er i tvivl om den dyreste altid vil være sammenlignen af de to eller hvordan?
# Jeg tror vi skal beholde begge, hvilket kan gøres ved at betinge på lotSize, når vi 
# fjerner dubletter. Der er enkelte som ligger med samme lotSize, jeg tror det er ægte dubletter.

# 7. Helårsgrunde: Ligner igen fejl. Ligner at den ældste række har de rigtige informationer.

# 8. Fritidsgrunde: Ligner igen fejl. Ligner at den ældste række har de rigtige informationer.
bolig_df_selected[bolig_df_selected['propertyType']==12]

Dubletter:0


,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate
24121,2122965,56.47018,9.39722,12,-,2600000,6.0,569,3913,NaN,0,Viborg,True,791,8800,Tagtækkervej 10,4569.0,9,0,0,0.0,Tagtækkervej,2024-07-04T09:07:06.273Z
36912,2131176,55.25401,9.48690,12,-,1950000,4.0,291,218,NaN,0,Haderslev,True,510,6100,Laurids Skaus Gade 5,6701.0,8,0,0,0.0,Laurids Skaus Gade,2024-08-05T13:07:11.000Z
37070,2132669,56.28728,9.31388,12,-,1050000,8.0,592,10530,NaN,0,Kjellerup,True,791,8620,Bødkersmindevej 2a,1773.0,9,0,0,NaN,Bødkersmindevej,2024-08-09T09:07:34.000Z
37482,2131489,55.29455,10.21436,12,-,630000,4.0,120,1133,NaN,0,Tommerup,True,420,5690,Knarreborgvej 8,5250.0,7,0,0,0.0,Knarreborgvej,2024-08-06T09:07:14.203Z


### Fjerner dubletter

In [14]:
# Sort by 'createdDate' in ascending order to ensure the oldest observations are kept
bolig_df_selected = bolig_df_selected.sort_values(by='createdDate')

# Split the dataframe based on PropertyType
df_type1 = bolig_df_selected[bolig_df_selected['propertyType'] == 1]
df_type2 = bolig_df_selected[bolig_df_selected['propertyType'] == 2]
df_type3 = bolig_df_selected[bolig_df_selected['propertyType'] == 3]
# df_type4 = bolig_df_selected[bolig_df_selected['propertyType'] == 4]
df_type6 = bolig_df_selected[bolig_df_selected['propertyType'] == 6]
df_type7 = bolig_df_selected[bolig_df_selected['propertyType'] == 7]
# df_type8 = bolig_df_selected[bolig_df_selected['propertyType'] == 8]
df_type9 = bolig_df_selected[bolig_df_selected['propertyType'] == 9]
df_type10 = bolig_df_selected[bolig_df_selected['propertyType'] == 10]
df_type11 = bolig_df_selected[bolig_df_selected['propertyType'] == 11]
df_type12 = bolig_df_selected[bolig_df_selected['propertyType'] == 12]


In [15]:
# Drop duplicates for PropertyType 1
df_type1 = df_type1.drop_duplicates(subset=['street', 'zipCode'])

# Drop duplicates for PropertyType 2
df_type2 = df_type2.drop_duplicates(subset=['street', 'zipCode', 'size'])

# Drop duplicates for PropertyType 3
df_type3 = df_type3.drop_duplicates(subset=['street', 'zipCode', 'size'])

# Drop duplicates for PropertyType 6
df_type6 = df_type6.drop_duplicates(subset=['street', 'zipCode', 'lotSize'])

# Drop duplicates for PropertyType 7
df_type7 = df_type7.drop_duplicates(subset=['street', 'zipCode'])


In [16]:
# Concatenate the filtered dataframes back together
bolig_df_selected2 = pd.concat([df_type1, df_type2, df_type3, df_type6, df_type7, df_type9, df_type10,
                               df_type11, df_type12])

# Reset index if needed
bolig_df_selected2.reset_index(drop=True, inplace=True)

print(len(bolig_df_selected2))

47142


In [17]:
bolig_df_selected2[bolig_df_selected2['street']=="Danehøje 16A"]
bolig_df_selected2[bolig_df_selected2['street']=="Langsand 218"]

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate


## EnergyClass
Sørger for at alt er med småt og retter mærkelige energiklasser

In [18]:
# Retter alt til stort
bolig_df_selected2['energyClass']= bolig_df_selected2['energyClass'].str.upper()

In [19]:
# Ændre mærkelige energimærker
# Define the mapping of old values to new values
mapping = {
    'M': 'A2',
    'K': 'A10',
    'J': 'A15',
    'I': 'A20'
}

# Replace the values in the 'energyClass' column
bolig_df_selected2['energyClass'] = bolig_df_selected2['energyClass'].replace(mapping)
bolig_df_selected2[bolig_df_selected2['energyClass']=='A20']

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate
230,1709351,57.16998,9.74983,1,A20,3850000,5.0,182,924,NaN,2021,Aabybro,False,849,9440,Solskins Alle 9,21153.0,11,2443,0,NaN,Solskins Alle,2020-09-28T23:58:22.123Z
355,1773472,57.45257,9.98009,1,A20,9000000,7.0,374,1898,NaN,2012,Hjørring,False,860,9800,Søndervang 5,24064.0,11,4765,114,0.0,Søndervang,2021-04-20T00:17:27.260Z
585,1837264,56.57827,9.00978,1,A20,3365000,5.0,160,1203,NaN,2023,Skive,False,779,7800,"Vindevej 103, Vinde",21031.0,9,2157,0,NaN,Vindevej,2021-10-15T00:05:14.840Z
596,1839481,56.57799,9.00993,1,A20,3695000,5.0,166,1203,NaN,2023,Skive,False,779,7800,"Vindevej 101, Vinde",22259.0,9,2517,0,NaN,Vindevej,2021-10-23T00:00:09.130Z
714,1860360,55.45981,9.10563,1,A20,2995000,4.0,129,339,NaN,2024,Vejen,False,575,6600,"Kongstedgårds Alle 16F, Askov",23217.0,8,2425,0,0.0,Kongstedgårds Alle,2022-01-27T01:10:44.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42622,2120539,55.80697,8.68127,6,A20,4498000,7.0,180,11000,NaN,2008,Ølgod,False,573,6870,Agersnapvej 24,24988.0,8,2435,0,0.0,Agersnapvej,2024-06-27T00:06:22.563Z
42642,2121141,55.68951,11.79062,6,A20,17995000,4.0,354,84281,NaN,1908,Holbæk,False,316,4300,"Bredetvedvej 23, Bredetved",50833.0,6,2969,0,0.0,Bredetvedvej,2024-06-28T22:05:25.000Z
42702,2123546,55.92217,12.17867,6,A20,39000000,10.0,383,233211,NaN,2016,Skævinge,False,219,3320,Lindebjergvej 8,101827.0,3,11062,181,NaN,Lindebjergvej,2024-07-05T22:04:31.310Z
42790,2126184,55.55734,12.11298,6,A20,10995000,8.0,162,97916,NaN,1877,Havdrup,False,265,4622,Bulbrovej 1,67870.0,5,4383,0,0.0,Bulbrovej,2024-07-15T22:35:01.670Z


### Rooms
Sætter boliger med 0 værelser til missing, hvis det ikke er en grund

In [20]:
# Set 'rooms' to NaN where 'rooms' == 0 and 'propertyType' is not 7 or 8
bolig_df_selected2.loc[(bolig_df_selected2['rooms'] == 0) & 
                      (~bolig_df_selected2['propertyType'].isin([7, 8])), 'rooms'] = np.nan

### Size
Overvej:
1. Skal vi fjerne ejendomme med size under 10 kvm, som ikke er grunda, da det ligner erhvervsejendomme?
2. Skal vi fjerne de to garager jeg har identificeret?

### lotSize
Overvej:
1. Sæt lotSize til missing, hvis den er 0 for lejligheder?
2. Skal vi gøre noget ved høj lotSize for fritidsejendomme?

In [21]:
# Sætter lotSize til missing, hvis den er 0 eller 1  og det ikke er en lejlighed
bolig_df_selected2.loc[
    (bolig_df_selected2['lotSize'].isin([0,1])) & 
    (bolig_df_selected2['propertyType'] != 3), 'lotSize'
] = np.nan

bolig_df_selected2[bolig_df_selected2['propertyType']!=3].sort_values('lotSize')

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate
31106,2059729,55.26901,9.89621,2,C,695000,3.0,60,43.0,NaN,1830,Assens,False,420,5610,Kindhestegade 16,11583.0,7,842,0,0.0,Kindhestegade,2023-12-08T09:39:28.550Z
31421,2078184,56.46053,10.03292,2,E,1395000,3.0,95,45.0,NaN,1900,Randers C,False,730,8900,Von Hattenstræde 5,14684.0,10,1686,0,0.0,Von Hattenstræde,2024-03-04T10:31:57.990Z
30630,1881317,55.26896,9.89483,2,D,545000,3.0,63,45.0,NaN,1800,Assens,False,420,5610,Adelgade 15,8650.0,7,1212,11,0.0,Adelgade,2022-04-05T00:05:47.383Z
32660,2115321,56.08740,8.24349,2,C,995000,2.0,41,47.0,NaN,1760,Ringkøbing,False,760,6950,Ø Strandgade 43,24268.0,9,1223,0,0.0,Ø Strandgade,2024-06-15T00:18:40.340Z
32566,2113016,55.86315,9.84703,2,C,1495000,3.0,74,47.0,NaN,1880,Horsens,False,615,8700,Grønnegade 21,20202.0,10,1251,0,0.0,Grønnegade,2024-06-09T00:29:33.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47098,2129067,55.47488,8.44896,9,E,1498000,4.0,92,NaN,1.0,1934,Esbjerg,False,561,6700,"Svendsgade 68, 1.",16282.0,8,2247,0,0.0,Svendsgade,2024-07-26T23:50:12.610Z
47101,2129392,54.85567,10.52046,9,G,650000,3.0,68,NaN,1.0,1887,Marstal,False,492,5960,"Havnegade 6, 1.",9558.0,7,938,0,0.0,Havnegade,2024-07-29T18:33:28.160Z
47117,2131932,55.64644,12.10627,9,E,2250000,4.0,81,NaN,NaN,1955,Roskilde,False,265,4000,"Mågevej 32, st. tv.",27777.0,5,2629,0,0.0,Mågevej,2024-08-07T22:05:05.863Z
47121,2132555,56.14823,10.19481,9,C,2698000,2.0,60,NaN,NaN,1897,Aarhus C,False,751,8000,"Wilstersgade 20, st..",44966.0,10,2573,0,0.0,Wilstersgade,2024-08-08T22:12:25.040Z


### buildYear

In [22]:
# Sætter byggeår til missing, hvis den er fra før 1575
bolig_df_selected2.loc[(bolig_df_selected2['buildYear'] < 1575), 'buildYear'] = np.nan
bolig_df_selected2[(bolig_df_selected2['buildYear'] < 1575)]

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate


In [23]:
# Sætter byggeår til missing, hvis det er en grund
bolig_df_selected2.loc[(bolig_df_selected2['propertyType'].isin([7,8]), 'buildYear')] = np.nan
bolig_df_selected2[(~bolig_df_selected2['buildYear'].isna()) & (bolig_df_selected2['propertyType'].isin([7,8]))]

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate


### PropertyType
1. Lægger 10, 11 og 12 sammen til andet

In [24]:
# Replace propertyType 11 and 12 with 10
bolig_df_selected2['propertyType'] = bolig_df_selected2['propertyType'].replace([11, 12], 10)


### Municipality
1. Sætter kommune til missing, hvis den er 0 eller -1

In [25]:
bolig_df_selected2.loc[(bolig_df_selected2['municipality'].isin([0,-1])),
                       'municipality'] = np.nan
bolig_df_selected2.groupby('municipality').size()


municipality
101.0    1738
147.0     247
151.0     141
153.0      99
155.0     131
         ... 
840.0     497
846.0     663
849.0     696
851.0    1948
860.0    1032
Length: 98, dtype: int64

### Coordinates
Retter missing koordinater

In [26]:
# Sætter koordinater med 0 eller -1 til missing
import numpy as np

# Set 'longitude' to NaN where it is equal to -1 or 0
bolig_df_selected2.loc[
    (bolig_df_selected2['longitude'] == -1) | (bolig_df_selected2['longitude'] == 0), 
    'longitude'
] = np.nan

# Set 'latitude' to NaN where it is equal to -1 or 0
bolig_df_selected2.loc[
    (bolig_df_selected2['latitude'] == -1) | (bolig_df_selected2['latitude'] == 0), 
    'latitude'
] = np.nan

# Eksporterer data
Eksporterer cleaned data

In [27]:
bolig_df_selected2.to_csv(data_path/'boliger_cleaned.csv', sep=';', index=False, encoding='utf-8')

In [314]:
bolig_df_selected[bolig_df_selected['latitude']==0]

,id,latitude,longitude,propertyType,energyClass,price,rooms,size,lotSize,floor,buildYear,city,isForeclosure,municipality,zipCode,street,squaremeterPrice,area,exp,basementSize,businessArea,cleanStreet,createdDate
42386,2133485,0.0,0.0,1,C,3998000,9.0,210,421,NaN,1930,Åbyhøj,False,0,8230,"Tousvej 116, Gl. Åby",19038.0,0,4534,85,NaN,Tousvej,2024-08-12T23:30:10.280Z
44452,2133618,0.0,0.0,3,B,7995000,5.0,230,0,2.0,2006,Nyborg,False,450,5800,"Vesterhavnen 19, 2.",34760.0,7,9615,0,NaN,Vesterhavnen,2024-08-13T22:03:57.533Z
43332,2134040,0.0,0.0,3,b,9800000,6.0,167,0,1.0,2006,Charlottenlund,False,0,2920,Ordrup Jagtvej 20A 1 + 2,58682.0,0,7821,0,NaN,Ordrup Jagtvej,2024-08-14T13:39:50.173Z
33411,2134151,0.0,0.0,1,g,13995000,7.0,357,2772,NaN,1910,Aalborg,False,0,9000,Hasserisvej 134,39201.0,0,10223,205,NaN,Hasserisvej,2024-08-14T22:35:02.580Z
46866,2134204,0.0,0.0,1,D,1450000,5.0,180,552,NaN,1932,Thisted,False,0,7700,"Winthersmøllevej 19, Tingstrup",8055.0,0,1778,40,NaN,Winthersmøllevej,2024-08-14T23:30:03.003Z
